![Title](./image/title.png)

## Problem Scenario: Cryptocurrency Investing App

You're building a cryptocurrency investing app that needs to reuse a complex third-party library for buying/selling crypto.

**Challenges:**

*   **Code Duplication:**  Multiple parts of the app might need to use the same `buyCurrency` logic. Directly calling the library from each place leads to code duplication.
*   **Tight Coupling:** Direct interaction with the library makes the app's business logic tightly coupled to the library's implementation details. Any change in the library requires changes throughout the app.
*   **Complexity:** The library is complex and exposes too much unnecessary functionality to the rest of the application.  The application should ideally only need a simplified interface.
*   **Requirement Changes:** New requirements (e.g., checking user balance, sending email confirmations) would necessitate modifying the `buyCurrency` logic in every location it's used.

**Solution Needed:**

A way to centralize the `buyCurrency` logic, simplify the interaction with the third-party library, and isolate the rest of the application from the library's complexity, all while allowing for future modifications without widespread code changes. This is where the Façade pattern comes in.

### Before Facade Pattern

In [ ]:
class UIService:
    @staticmethod
    def get_logged_in_user_id():
        return "user123"

class DatabaseService:
    def get_user(self, user_id):
        user = User()
        user.id = user_id
        user.name = "John Doe"
        user.balance = 5000
        user.currency = "USD"
        user.account_nbr = "123456789"
        return user

class User:
    def __init__(self):
        self.id = None
        self.name = None
        self.balance = None
        self.currency = None
        self.account_nbr = None

class CryptoFactory:
    @staticmethod
    def get_crypto_service(currency):
        if currency == "BTC":
            return BitcoinService()
        elif currency == "ETH":
            return EthereumService()
        else:
            raise ValueError(f"Unsupported currency: {currency}")

class CryptoService:
    def __init__(self):
        self.crypto_database = None
        self.complex_stuff = None

    def buy_currency(self, user, amount):
        raise NotImplementedError("Subclasses must implement buy_currency")


class BitcoinService(CryptoService):
    def buy_currency(self, user, amount):
        print(f"Buying {amount} BTC for user {user.name} (ID: {user.id})")
        user.balance -= amount
        print(f"New balance: {user.balance}")

class EthereumService(CryptoService):
    def buy_currency(self, user, amount):
        print(f"Buying {amount} ETH for user {user.name} (ID: {user.id})")
        user.balance -= amount
        print(f"New balance: {user.balance}")

def main():
    db_service = DatabaseService()
    user = db_service.get_user(UIService.get_logged_in_user_id())
    CryptoFactory.get_crypto_service("BTC").buy_currency(user, 1000)

if __name__ == "__main__":
    main()

## Drawbacks of Direct Subsystem Interaction (Before Facade)

The code above demonstrates several issues that make it a poor design choice for larger applications, and that ultimately *force* us to consider the Facade pattern:

*   **Tight Coupling:** The `main` function is directly interacting with `UIService`, `DatabaseService`, `CryptoFactory`, and `BitcoinService`.  Any change in the interfaces or implementations of these components will require modifications in `main` or other code directly using them.

*   **Code Duplication Risk:**  If other parts of the application needed to perform the same user retrieval and crypto purchase logic, the code would be duplicated.  This violates the DRY (Don't Repeat Yourself) principle.

*   **Lack of Centralized Control:** There's no central place to add logic that is related to the crypto purchase process. Consider these scenarios:

    *   **Balance Check:** What if you need to check if the user has sufficient funds *before* initiating the crypto purchase?  You'd have to add that check in every location that uses this logic.
    *   **Post-Transaction Actions:** What if you need to send a confirmation email or log the transaction? Again, this needs to be replicated.
    *   **Transaction Rollback:** What if a step fails? Implementing a robust rollback mechanism becomes significantly more complex with distributed logic.

*   **Exposure of Implementation Details:** The `main` function needs to know how to get a user ID, how to fetch a user from the database, how to create a crypto service (using `CryptoFactory`), and how to execute the purchase.  This exposes too much of the internal workings of the system.  Clients shouldn't need to be aware of the complex object creation.

*   **Difficult to Test:**  Testing becomes more challenging because `main` is tightly coupled to these external services.  It's difficult to isolate the logic for unit testing.  You'd likely need to mock multiple classes to test the `main` function.

*   **Maintenance Nightmare:** Any change to the underlying services can potentially break multiple parts of the application that directly use them. This increases the risk of introducing bugs and makes maintenance more difficult.

*   **Library Versioning:** If you need to upgrade or switch the external CryptoFactory, DatabaseService libraries, you have to change every place in your code that uses it.

**In essence, the absence of a Facade leads to a brittle, difficult-to-maintain, and error-prone system.**  The Facade pattern provides a solution by encapsulating the complexity and offering a simplified interface.  The next step would be to introduce a class that acts as the "face" of the crypto purchase process, hiding the internal details and providing a clean abstraction.

![Example](./image/example.png)

In [ ]:
class UIService:
    @staticmethod
    def get_logged_in_user_id():
        return "user123"

class DatabaseService:
    def get_user(self, user_id):
        user = User()
        user.id = user_id
        user.name = "John Doe"
        user.balance = 5000
        user.currency = "USD"
        user.account_nbr = "123456789"
        return user

class User:
    def __init__(self):
        self.id = None
        self.name = None
        self.balance = None
        self.currency = None
        self.account_nbr = None

class CryptoFactory:
    @staticmethod
    def get_crypto_service(currency):
        if currency == "BTC":
            return BitcoinService()
        elif currency == "ETH":
            return EthereumService()
        else:
            raise ValueError(f"Unsupported currency: {currency}")

class CryptoService:
    def __init__(self):
        self.crypto_database = None
        self.complex_stuff = None

    def buy_currency(self, user, amount):
        raise NotImplementedError("Subclasses must implement buy_currency")


class BitcoinService(CryptoService):
    def buy_currency(self, user, amount):
        print(f"Buying {amount} BTC for user {user.name} (ID: {user.id})")
        user.balance -= amount
        print(f"New balance: {user.balance}")

class EthereumService(CryptoService):
    def buy_currency(self, user, amount):
        print(f"Buying {amount} ETH for user {user.name} (ID: {user.id})")
        user.balance -= amount
        print(f"New balance: {user.balance}")

class MailService:
    def send_confirmation_mail(self, user):
        print(f"Sending confirmation email to {user.name} at {user.account_nbr}@example.com")

class BuyCryptoFacade:
    def buy_cryptocurrency(self, amount, currency):
        db_service = DatabaseService()
        user = db_service.get_user(UIService.get_logged_in_user_id())

        if user.balance < amount:
            print("Insufficient balance to perform transaction")
            return

        crypto_service = CryptoFactory.get_crypto_service(currency)
        crypto_service.buy_currency(user, amount)

        mail_service = MailService()
        mail_service.send_confirmation_mail(user)

def main():
    buy_crypto = BuyCryptoFacade()
    buy_crypto.buy_cryptocurrency(1000, "BTC")

if __name__ == "__main__":
    main()

## Façade Design Pattern: Structure and Components

**Overview:** The Façade pattern provides a simplified interface to a complex subsystem (e.g., a library or framework).

**Components (Relating to the Crypto App Example):**

*   **Façade (BuyCryptoFacade):**
    *   A class that provides a high-level, simplified interface to the complex subsystem.
    *   The `BuyCryptoFacade` class is responsible for hiding the complexities of the crypto library from the client.
    *   It knows how to direct client requests, orchestrate the necessary steps, and handle the underlying system.
    *   **Important:**  Can become a "god object" if it grows too large and handles unrelated features.  Use additional facades to prevent this.

*   **Additional Facades:**
    *   Can be created to prevent a single façade from becoming too complex.
    *   Can be used by both clients and other facades.

*   **Complex Subsystem (Third-Party Crypto Library):**
    *   Consists of multiple classes and objects that perform specific tasks.
    *   The `BuyCryptoFacade` initializes these objects in the correct order and provides them with the necessary data.
    *   Subsystem classes are unaware of the façade's existence and operate independently.

*   **Client (Investing App Components):**
    *   Uses the façade to interact with the subsystem instead of directly accessing the complex subsystem classes.
    *   This simplifies the client's code and reduces its dependence on the subsystem's implementation details.